In [6]:
import pandas as pd

# Đọc CSV ban đầu và CSV mới
#csv_1 = pd.read_csv('./OANDA_BTCUSD, 15 (9).csv')
csv_2 = pd.read_csv('./data/OANDA_XAUUSDNEW_Historical.csv')
csv_1 = pd.read_csv('./data/OANDA_XAUUSD_1m_Historical.csv')


# Chèn nội dung của CSV mới vào CSV ban đầu

csv_ban_dau = pd.concat([csv_2, csv_1], ignore_index=True)

# Lưu lại vào file mới
csv_ban_dau.to_csv('./data/OANDA_XAUUSD_1m_Historical.csv', index=False)


In [5]:
import pandas as pd

# Đọc CSV ban đầu và CSV mới
csv_1 = pd.read_csv('./OANDA_XAUUSD, 1.csv')
csv_2 = pd.read_csv('./OANDA_XAUUSD, 1 (1).csv')
csv_3 = pd.read_csv('./OANDA_XAUUSD, 1 (2).csv')
csv_4 = pd.read_csv('./OANDA_XAUUSD, 1 (3).csv')
csv_5 = pd.read_csv('./OANDA_XAUUSD, 1 (4).csv')
csv_6 = pd.read_csv('./OANDA_XAUUSD, 1 (5).csv')
csv_7 = pd.read_csv('./OANDA_XAUUSD, 1 (6).csv')
csv_8 = pd.read_csv('./OANDA_XAUUSD, 1 (7).csv')
csv_9 = pd.read_csv('./OANDA_XAUUSD, 1 (8).csv')
csv_10 = pd.read_csv('./OANDA_XAUUSD, 1 (9).csv')
csv_11 = pd.read_csv('./OANDA_XAUUSD, 1 (10).csv')
csv_12 = pd.read_csv('./OANDA_XAUUSD, 1 (11).csv')
csv_13 = pd.read_csv('./OANDA_XAUUSD, 1 (12).csv')
csv_14 = pd.read_csv('./OANDA_XAUUSD, 1 (13).csv')
csv_15 = pd.read_csv('./OANDA_XAUUSD, 1 (14).csv')
csv_16 = pd.read_csv('./OANDA_XAUUSD, 1 (5).csv')
csv_17 = pd.read_csv('./OANDA_XAUUSD, 1 (16).csv')
csv_18 = pd.read_csv('./OANDA_XAUUSD, 1 (17).csv')
csv_19 = pd.read_csv('./OANDA_XAUUSD, 1 (18).csv')

# Chèn nội dung của CSV mới vào CSV ban đầu

csv_ban_dau = pd.concat([

    csv_19,
    csv_18,
    csv_17,
    csv_16,
    csv_15, 
    csv_14, 
    csv_13,
    csv_12, 
    csv_11, 
    csv_10, 
    csv_9,
    csv_8,
    csv_7,
    csv_6,
    csv_5,
    csv_4,
    csv_3,
    csv_2,
    csv_1,
    ], ignore_index=True
)

# Lưu lại vào file mới
csv_ban_dau.to_csv('./data/OANDA_XAUUSDNEW_Historical.csv', index=False)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
import pickle

# Định nghĩa các cột cần sử dụng
columns = ["close", "open", "high", "low"]

# Định nghĩa các đường dẫn tới file dữ liệu
file_paths = [
    './train-data/OANDA_XAUUSD_Historical_Updated.csv',
]

# Khởi tạo scaler cho từng cột
scalers = {}
scalers['Price'] = MinMaxScaler(feature_range=(0, 1))

# Tạo danh sách để lưu trữ dữ liệu từ tất cả các file
all_data = []

# Load và thu thập dữ liệu từ tất cả các file
for file in file_paths:
    data = pd.read_csv(file)
    data = data[columns].dropna()
    all_data.append(data)

# Kết hợp tất cả dữ liệu lại
combined_data = pd.concat(all_data, axis=0)

df_melted = combined_data.melt(var_name='Type', value_name='Value')

scalers['Price'].fit(df_melted['Value'].values.reshape(-1, 1))
    

# Lưu các scaler vào file
with open('./scalers/scalers_1.pkl', 'wb') as f:
    pickle.dump(scalers, f)

print("Scalers have been fitted and saved successfully.")


In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, LayerNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
import pickle


columns = ["body", "STOCH_D", "STOCH_K"]

# Load and prepare data
data = pd.read_csv('./train-data/OANDA_XAUUSD_Historical.csv')
features = data[columns].values
scaled_data = np.zeros_like(features).astype(float)
# Initialize and fit scalers
scalers = {}
# Scale the features
for i, column in enumerate(columns):
    scalers[column] = MinMaxScaler(feature_range=(0, 1))
    scaled_data[:, i] = scalers[column].fit_transform(features[:, i].reshape(-1, 1)).flatten()
# Create the sliding window dataset
def create_multifeature_dataset(dataset, time_step=60):
    X, Y = [], []
    for i in range(len(dataset) - time_step - 1):
        X.append(dataset[i:(i + time_step), :])
        Y.append(dataset[i + time_step, 0])  
    return np.array(X), np.array(Y)

time_step = 20
X, y = create_multifeature_dataset(scaled_data, time_step)

# Adjust input shape to include features
num_features = X.shape[2]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Build the LSTM model
model = Sequential([
    Input(shape=(time_step, num_features)), 
    LSTM(300, return_sequences=True),
    LSTM(300, return_sequences=False),
    Dropout(0.2),
    Dense(1)
])

print(scaled_data)
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.5), loss='mean_squared_error')

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=64, verbose=1, epochs=1000, callbacks=[early_stopping])

#Save the model and scalers
with open('./model/predict_scalers_XAUUSD.pkl', 'wb') as f:
    pickle.dump(scalers, f)

model.save('./model/predict_model_XAUUSD.h5')


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import joblib
import ta
symbol = 'BTCUSDT'
# df = pd.read_csv('./train-data/OANDA_XAUUSD_Historical.csv')
df = pd.read_csv(f'./train-data/OANDA_XAUUSD_Historical.csv')
range_value = 15
# Xử lý giá trị thiếu
df.dropna(inplace=True)
lags = range(0, range_value)
features = [f'body_{i}' for i in lags]
features += [f'shadow_top_{i}' for i in lags]
features += [f'shadow_bottom_{i}' for i in lags]
# features += [f'volume_avg_{i}' for i in lags]

# Chọn đặc trưng và biến mục tiêu
X = df[features]
y = df['target']

# Chia dữ liệu
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=False)

# Khởi tạo và huấn luyện mô hình
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train, y_train)

# Dự đoán và đánh giá
y_pred = rf_classifier.predict(X_test)

# Dự đoán giá tiếp theo
latest_data = X.iloc[-1].values.reshape(1, -1)
next_movement = rf_classifier.predict(latest_data)
model_filename = f'./random_forest_xau_model.pkl'


joblib.dump(rf_classifier, model_filename)
if next_movement[0] > 0:
    print("Giá dự kiến sẽ TĂNG vào ngày tiếp theo.")
else:
    print("Giá dự kiến sẽ GIẢM vào ngày tiếp theo.")

KeyboardInterrupt: 

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, LayerNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
import pickle
from sklearn.preprocessing import MinMaxScaler
columns = ["close", "high", "low", "open"]

all_X, all_y = [], []

# Create the sliding window dataset with multiple features
def create_multifeature_dataset(dataset, time_step=60):
    X, Y = [], []
    for i in range(len(dataset) - time_step - 1):
        X.append(dataset[i:(i + time_step), :])
        Y.append(dataset[i + time_step, 0])
    return np.array(X), np.array(Y)

with open('./scalers/scalers_1.pkl', 'rb') as f:
    scalers = pickle.load(f)

# scalers = {}
# scalers['Price'] = MinMaxScaler(feature_range=(0, 1))
time_step = 60

file_paths = [
    './data/OANDA_XAUUSD_Historical.csv',
    './data/OANDA_XAUUSD_Historical_2.csv',
    './data/OANDA_XAUUSD_Historical_3.csv',
]

for file in file_paths:
    data = pd.read_csv(file)
    data = data[columns].dropna()
    features = data.values

    scaled_data = np.zeros_like(features).astype(float)

    for i, column in enumerate(columns):
        scaled_data[:, i] = scalers['Price'].transform(features[:, i].reshape(-1, 1)).flatten()

    X, y = create_multifeature_dataset(scaled_data, time_step)

    all_X.append(X)
    all_y.append(y)

X_combined = np.concatenate(all_X, axis=0)
Y_combined = np.concatenate(all_y, axis=0)
X_train, X_test, y_train, y_test = train_test_split(X_combined, Y_combined, test_size=0.2, random_state=42)
num_features = X_train.shape[2]

model = Sequential([
    Input(shape=(time_step, num_features)),
    LSTM(200, return_sequences=True),
    LayerNormalization(), 
    LSTM(200, return_sequences=False),
    LayerNormalization(), 
    Dropout(0.2),
    Dense(1)
])

print(scaled_data)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=0.0001)
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

# Huấn luyện mô hình
model.fit(X_train, y_train, validation_data=(X_test, y_test),callbacks=[early_stopping], batch_size=60, verbose=1, epochs=1000)

# Lưu mô hình
model.save('./predict_model_fx_3.h5')


In [ ]:
import pandas as pd
import ta
import ta.momentum
import ta.trend
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import ta.volatility
import ta.volume
# Đọc dữ liệu từ file CSV
data = pd.read_csv('./data/XAUUSD/data43_updated.csv')

# Tạo cột 'Next_Close' bằng cách dịch chuyển cột 'Close' lên trên một hàng
RSI = ta.momentum.RSIIndicator(data['Close'])
MACD = ta.trend.MACD(data['Close'])
STOCH = ta.momentum.StochasticOscillator(data['High'], data['Low'], data['Close'])
ROC = ta.momentum.ROCIndicator(close=data['Close'], window=12)
# SMA = ta.trend.SMAIndicator(close=data['Close'], window=60)
ATR = ta.volatility.AverageTrueRange(high=data['High'], low=data['Low'], close=data['Close'], window=14)

# data['RSI'] = RSI.rsi()
# data['MACD'] = MACD.macd()
# data['STOCH_K'] = STOCH.stoch()
# data['STOCH_D'] = STOCH.stoch_signal()
# data['ROC_RATE'] = ROC.roc()
# data['ATR'] = ATR.average_true_range()
data['isIncrease'] = data.apply(lambda row: 1 if row['Close'] > row['Open'] else 0, axis=1)
increment = 200
data['Close'] = data['Close'] + increment
data['High'] = data['High'] + increment
data['Low'] = data['Low'] + increment
data['Open'] = data['Open'] + increment


# columns = ["time","Open","High","Low","Close","Volume","RSI","MACD","MACD_SIGNAL","MACD_DIFF","isIncrease"]
# Lưu dữ liệu đã xử lý vào file CSV mới
output_path = './data/XAUUSD/data43_updated.csv'
data.to_csv(output_path, index=False)

In [ ]:
import pandas as pd
import ta
import ta.momentum
import ta.trend
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import ta.volatility
import ta.volume
# Đọc dữ liệu từ file CSV
data = pd.read_csv('./data/OANDA_XAUUSD_Historical.csv')

increment = 1.4

# Chuyển đổi các cột thành số thập phân, nhân với 1.5 và làm tròn đến 1 chữ số thập phân
data['close'] = np.round(data['close'].astype(float) * increment, 4)
data['low'] = np.round(data['low'].astype(float) * increment, 4)
data['high'] = np.round(data['high'].astype(float) * increment, 4)
data['open'] = np.round(data['open'].astype(float) * increment, 4)

# columns = ["time","Open","High","Low","Close","Volume","RSI","MACD","MACD_SIGNAL","MACD_DIFF","isIncrease"]
# Lưu dữ liệu đã xử lý vào file CSV mới
output_path = './data/OANDA_XAUUSD_Historical_3.csv'
data.to_csv(output_path, index=False)


# Tuning Model

In [ ]:
from tensorflow.keras.models import load_model # type: ignore
from tensorflow.keras.callbacks import EarlyStopping # type: ignore
import pickle
from tensorflow.keras.optimizers import Adam # type: ignore
from sklearn.model_selection import train_test_split # type: ignore
import numpy as np
import pandas as pd
from tensorflow.keras.callbacks import ReduceLROnPlateau # type: ignore
# Load the model and scalers
model = load_model('./predict_model_fx_4.h5')

with open('./scalers/scalers_1.pkl', 'rb') as f:  
    scalers = pickle.load(f) 

columns = ["close", "high", "low", "open"]

data = pd.read_csv('./data/OANDA_XAUUSD_Historical_Updated.csv')
data = data[columns].dropna()
features = data.values
scaled_data = np.zeros_like(features).astype(float)

# Scale other features separately
for i, column in enumerate(columns):
    scaled_data[:, i] = scalers['Price'].transform(features[:, i].reshape(-1, 1)).flatten()

def create_multifeature_dataset(dataset, time_step=60):
    X, Y = [], []
    for i in range(len(dataset) - time_step - 1):
        X.append(dataset[i:(i + time_step), :])
        Y.append(dataset[i + time_step, 0])  
    return np.array(X), np.array(Y)

time_step = 14
X, y = create_multifeature_dataset(scaled_data, time_step)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_squared_error')
# Huấn luyện mô hình
model.fit(X_train, y_train, validation_data=(X_test, y_test), callbacks=[early_stopping], batch_size=48, verbose=1, epochs=100000)

model.save('./predict_model_fx_5.h5')


In [2]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors

import joblib
range_value = 10
time = 15
# Đọc dữ liệu từ các file CSV
df_historical = pd.read_csv(f'./train-data/OANDA_XAUUSD_1m_Historical.csv')
window_size = 10
columns = ['stoch']

# Tạo segments cho cả 'body' và 'RSI'
def create_segments(data):
    return np.array([
        np.hstack([data[col].values[i:i + window_size] for col in columns])
        for i in range(len(data) - window_size + 1)
    ])

# Tạo segments cho indicator
historical_segments = create_segments(data=df_historical)

# Khởi tạo mô hình NearestNeighbors với hàm khoảng cách 'euclidean'
nearest_neighbors = NearestNeighbors(n_neighbors=1, algorithm='auto', metric='euclidean')

# Huấn luyện mô hình với dữ liệu historical_segmentsz
nearest_neighbors.fit(historical_segments)

# Lưu mô hình đã huấn luyện ra file để sử dụng lại sau này
model_filename = f'nearest_neighbors_model.joblib'
joblib.dump(nearest_neighbors, model_filename)

print(f"Model saved as {model_filename}")


KeyboardInterrupt: 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import joblib

time = 5
# Đọc dữ liệu từ các file CSV
df_historical = pd.read_csv(f'./OANDA_XAUUSD_{time}m_Historical.csv')
window_size = 1
indicators = [
   
]
range_value = 10

# Khởi tạo mô hình NearestNeighbors với hàm khoảng cách tùy chỉnh
nearest_neighbors = NearestNeighbors(n_neighbors=1, algorithm='auto', metric='euclidean')

# Tạo segments cho cả 'body' và 'RSI'
def create_segments(data, indicator):
    columns = [f'{indicator}_{i}' for i in range(range_value)]  

    return np.array([
        np.hstack([data[col].values[i:i + window_size] for col in columns])
        for i in range(len(data) - window_size + 1)
    ])
for indicator in indicators:
    print(f"Training model for indicator: {indicator}")

    # Tạo segments cho indicator
    historical_segments = create_segments(data=df_historical, indicator=indicator)

    print(historical_segments)

    # Khởi tạo mô hình NearestNeighbors với hàm khoảng cách 'euclidean'
    nearest_neighbors = NearestNeighbors(n_neighbors=1, algorithm='auto', metric='euclidean')

    # Huấn luyện mô hình với dữ liệu historical_segments
    nearest_neighbors.fit(historical_segments)

    # Lưu mô hình đã huấn luyện ra file để sử dụng lại sau này
    model_filename = f'nearest_neighbors_model_XAUUSD_{indicator}.joblib'
    joblib.dump(nearest_neighbors, model_filename)

    print(f"Model for {indicator} saved as {model_filename}")


In [ ]:
# transformer_forex_prediction.py

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler

# 1. Đọc và tiền xử lý dữ liệu

# Đọc dữ liệu từ tệp CSV
df = pd.read_csv('train-data/OANDA_XAUUSD_15m_Historical.csv')

# Sử dụng giá đóng cửa
data = df['STOCH'].values

# Tiền xử lý dữ liệu
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data.reshape(-1, 1))

# 2. Tạo dữ liệu huấn luyện và kiểm tra

sequence_length = 60  # Số bước thời gian để dự đoán

# Chia dữ liệu thành tập huấn luyện và kiểm tra
train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size - sequence_length:]

# Hàm tạo dữ liệu chuỗi thời gian
def create_sequences(data, seq_length):
    X = []
    y = []
    for i in range(seq_length, len(data)):
        X.append(data[i - seq_length:i])
        y.append(data[i])
    return np.array(X), np.array(y)

# Tạo dữ liệu huấn luyện
X_train, y_train = create_sequences(train_data, sequence_length)

# Tạo dữ liệu kiểm tra
X_test, y_test = create_sequences(test_data, sequence_length)

# Thêm chiều cho dữ liệu (nếu cần)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# 3. Xây dựng mô hình Transformer

# Hàm tạo lớp Positional Encoding
def positional_encoding(position, d_model):
    angle_rads = np.arange(position)[:, np.newaxis] / np.power(
        10000, (2 * (np.arange(d_model)[np.newaxis, :] // 2)) / np.float32(d_model)
    )
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])  # Áp dụng sin cho các vị trí chẵn
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])  # Áp dụng cos cho các vị trí lẻ
    pos_encoding = angle_rads[np.newaxis, ...]
    return tf.cast(pos_encoding, dtype=tf.float32)

# Hàm tạo lớp Transformer Block
def transformer_block(inputs, num_heads, ff_dim):
    attention_output = keras.layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=inputs.shape[-1]
    )(inputs, inputs)
    attention_output = keras.layers.Dropout(0.1)(attention_output)
    out1 = keras.layers.LayerNormalization(epsilon=1e-6)(inputs + attention_output)
    ffn_output = keras.layers.Dense(ff_dim, activation='relu')(out1)
    ffn_output = keras.layers.Dense(inputs.shape[-1])(ffn_output)
    ffn_output = keras.layers.Dropout(0.1)(ffn_output)
    return keras.layers.LayerNormalization(epsilon=1e-6)(out1 + ffn_output)

# Định nghĩa các tham số mô hình
d_model = 64
num_heads = 4
ff_dim = 64

# Xây dựng mô hình
inputs = keras.layers.Input(shape=(sequence_length, 1))
x = keras.layers.Dense(d_model)(inputs)

# Thêm Positional Encoding
pos_encoding = positional_encoding(sequence_length, d_model)
x += pos_encoding[:, :sequence_length, :]

# Thêm Transformer Block
x = transformer_block(x, num_heads, ff_dim)

# Flatten và đưa qua Dense Layer
x = keras.layers.GlobalAveragePooling1D()(x)
x = keras.layers.Dropout(0.1)(x)
x = keras.layers.Dense(64, activation='relu')(x)
x = keras.layers.Dense(1)(x)

outputs = x

model = keras.Model(inputs=inputs, outputs=outputs)

# 4. Biên dịch và huấn luyện mô hình

model.compile(optimizer='adam', loss='mean_squared_error')

# Huấn luyện mô hình
history = model.fit(
    X_train,
    y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_test, y_test)
)

# 5. Dự đoán và đánh giá mô hình

# Dự đoán trên tập kiểm tra
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
y_test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))

# Save the model and scalers
with open('./model/predict_scalers_XAUUSD.pkl', 'wb') as f:
    pickle.dump(scaler, f)

model.save('./model/predict_model_XAUUSD.h5')



# Preprocessing Data

In [7]:
import pandas as pd
import ta
import ta.momentum
import ta.trend
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import ta.trend
import ta.volatility
import ta.volume
time = 240
round_num = 5
range_value = 10
# Đọc dữ liệu từ file CSV
new_data = pd.read_csv(f'./data/OANDA_XAUUSD_{time}m_Historical.csv')

new_data['body'] =  np.round(new_data['close'] - new_data['open'], round_num)
new_data['shadow'] = np.round(new_data['high'] - new_data['low'], round_num)
new_data['target'] = np.where(new_data['close'].shift(-1) > new_data['close'], 1, 0)

# # Tạo cột 'Next_Close' bằng cách dịch chuyển cột 'Close' lên trên một hàng
STOCH = ta.momentum.StochasticOscillator(new_data['high'], new_data['low'], new_data['close'], fillna=True)
TSI = ta.momentum.TSIIndicator(close=new_data['close'])
RSI = ta.momentum.RSIIndicator(close=new_data['close'])
STOCH_RSI = ta.momentum.StochRSIIndicator(close=new_data['close'])
PPO = ta.momentum.PercentagePriceOscillator(close=new_data['close'])
PVO = ta.momentum.PercentageVolumeOscillator(volume=new_data['Volume'])
UO = ta.momentum.UltimateOscillator(high=new_data['high'], low=new_data['low'], close=new_data['close'])
WR = ta.momentum.WilliamsRIndicator(high=new_data['high'], low=new_data['low'], close=new_data['close'])
ROC = ta.momentum.ROCIndicator(close=new_data['close'])
AO = ta.momentum.AwesomeOscillatorIndicator(high=new_data['high'], low=new_data['low'])

# Tạo các cột indicator
new_data['STOCH_RSI'] = np.round(STOCH_RSI.stochrsi(), round_num)
new_data['TSI'] = np.round(TSI.tsi(), round_num)
new_data['RSI'] = np.round( RSI.rsi(), round_num)
new_data['STOCH'] = np.round(STOCH.stoch(), round_num)


lags = range(0, range_value)

columns = [new_data['STOCH'].shift(i).rename(f'STOCH_{i}') for i in lags]
df = pd.concat(columns, axis=1)
new_data = new_data.join(df)

# Kết hợp DataFrame mới với DataFrame gốc
columns = [new_data['TSI'].shift(i).rename(f'TSI_{i}') for i in lags]
df = pd.concat(columns, axis=1)
new_data = new_data.join(df)

# Kết hợp DataFrame mới với DataFrame gốc
columns = [new_data['RSI'].shift(i).rename(f'RSI_{i}') for i in lags]
df = pd.concat(columns, axis=1)
new_data = new_data.join(df)

# Kết hợp DataFrame mới với DataFrame gốc
columns = [new_data['STOCH_RSI'].shift(i).rename(f'STOCH_RSI_{i}') for i in lags]
df = pd.concat(columns, axis=1)
new_data = new_data.join(df)

# Lưu dữ liệu đã xử lý vào file CSV mới
output_path = f'./train-data/OANDA_XAUUSD_{time}m_Historical.csv'
new_data.dropna().to_csv(output_path, index=False)

In [10]:
import pandas as pd
import numpy as np
from ta import add_all_ta_features
from ta.utils import dropna
time = 5
round_num = 5
range_value = 10
# Đọc dữ liệu từ file CSV
new_data = pd.read_csv(f'./data/OANDA_XAUUSD_{time}m_Historical.csv', sep=',')
new_data = dropna(new_data)
new_data = add_all_ta_features(df=new_data, open="open", high="high", low="low", close="close", volume="Volume", fillna=True)
new_data['target'] = np.where(new_data['close'].shift(-1) > new_data['close'], 1, 0)
# Lưu dữ liệu đã xử lý vào file CSV mới
output_path = f'./OANDA_XAUUSD_{time}m_Historical.csv'
new_data.to_csv(output_path, index=False)

In [ ]:
import pandas as pd
import numpy as np

# Thiết lập các tham số
time = 15
round_num = 5
indicator = 'DPO'

# Đọc dữ liệu từ file CSV
new_data = pd.read_csv(f'./data_XAUUSD.csv')

# Tạo các cột 'body', 'shadow', 'target'
new_data['body'] = np.round(new_data['close'] - new_data['open'], round_num)
new_data['shadow'] = np.round(new_data['high'] - new_data['low'], round_num)
new_data['target'] = np.where(new_data['close'].shift(-1).rolling(-1) > new_data['close'], 1, 0)

# Tạo các cột lag từ 'close_1' đến 'close_49' (tổng cộng 49 cột)
lags = range(0, 50)
lagged_data = [new_data['close'].shift(i).rename(f'close_{i}') for i in lags]
lagged_df = pd.concat(lagged_data, axis=1)

# Kết hợp DataFrame mới với DataFrame gốc
new_data = new_data.join(lagged_df)

# Xác định danh sách các cột 'close' cần chuẩn hóa
close_columns = [f'close_{i}' for i in lags] + ['close']  # Đưa 'close' vào cuối để tránh trùng lặp với 'close_0'

# Chuẩn hóa Min-Max theo từng hàng bằng vectorization
# Tính min và max cho mỗi hàng
row_min = new_data[close_columns].min(axis=1)
row_max = new_data[close_columns].max(axis=1)
range_val = row_max - row_min

# Tránh chia cho 0 bằng cách thay thế range_val bằng 1 ở những hàng range_val = 0
range_val_replaced = range_val.replace(0, 1)

# Thực hiện chuẩn hóa
normalized_df = (new_data[close_columns].subtract(row_min, axis=0)).divide(range_val_replaced, axis=0)

# Đổi tên các cột đã chuẩn hóa
normalized_df.columns = [f'normalized_{col}' for col in close_columns]

# Kết hợp DataFrame gốc với DataFrame đã chuẩn hóa
df_final = pd.concat([new_data, normalized_df], axis=1)

# Xóa các dòng chứa NaN do tạo lag và chuẩn hóa
df_final_clean = df_final.dropna()

# Lưu dữ liệu đã xử lý vào file CSV mới
output_path = f'./train-data/OANDA_XAUUSD_{time}m_Historical_v2.csv'
df_final_clean.to_csv(output_path, index=False)

print(f"Chuẩn hóa hoàn thành và lưu vào: {output_path}")


In [1]:
import pandas as pd
import numpy as np
import ta
import ta.momentum
import ta.trend
import ta.volume

def calculate_shadow_top(row):
    if row['close'] > row['open']:  # Nến tăng
        return row['high'] - row['close']
    else:  # Nến giảm
        return row['high'] - row['open']

def calculate_shadow_bottom(row):
    if row['close'] > row['open']:  # Nến tăng
        return row['open'] - row['low']
    else:  # Nến giảm
        return row['close'] - row['low']


# Thiết lập các tham số
time = 15
round_num = 5
# Đọc dữ liệu từ file CSV
new_data = pd.read_csv(f'./data/OANDA_XAUUSD_1m_Historical.csv')

# Tạo các cột 'body', 'shadow', 'target'
new_data['body'] = np.round(new_data['close'] - new_data['open'], round_num)

# new_data['shadow_top'] = np.where(new_data['body'] > 0, np.round(new_data['high'] - new_data['low'], round_num), -(np.round(new_data['high'] - new_data['low'], round_num)))
new_data['shadow_top'] = new_data.apply(calculate_shadow_top, axis=1)
new_data['shadow_bottom'] = new_data.apply(calculate_shadow_bottom, axis=1)
new_data['target'] = np.where(new_data['body'].shift(-14).rolling(window=15).mean() >= 0, 1, 0)
# new_data['volume_avg'] = new_data['Volume'].rolling(window=15).mean()

lags = range(0, 15)
lagged_data = [new_data['body'].shift(i).rename(f'body_{i}') for i in lags]
lagged_data += [new_data['shadow_top'].shift(i).rename(f'shadow_top_{i}') for i in lags]
lagged_data += [new_data['shadow_bottom'].shift(i).rename(f'shadow_bottom_{i}') for i in lags]
# lagged_data += [new_data['volume_avg'].shift(i).rename(f'volume_avg_{i}') for i in lags]

lagged_df = pd.concat(lagged_data, axis=1)

# Kết hợp DataFrame mới với DataFrame gốc
new_data = new_data.join(lagged_df)

# Lưu dữ liệu đã xử lý vào file CSV mới
output_path = f'./train-data/OANDA_XAUUSD_Historical.csv'
new_data.dropna().to_csv(output_path, index=False)

print(f"Chuẩn hóa hoàn thành và lưu vào: {output_path}")


Chuẩn hóa hoàn thành và lưu vào: ./train-data/OANDA_XAUUSD_Historical.csv
